# Connexion_France_Travail_ADZUNA.ipynb

Ce script a pour objectif d'extraire les offres d'emploi mises à disposition par l'API de France Travail et de l'API ADZUNA et de les stocker dans un fichier (CSV ou Excel) en local.

Comment ?
1. Sur la base de critères spécifiques (mots clés, localisation, etc...), 
    - lancement d'une requête pour obtenir les offres d'emploi correspondantes via l'API France Travail
    - lancement d'une requête pour obtenir les offres d'emploi correspondantes via l'API Adzuna
2. Une fois les offres trouvées, vérification et suppression des doublons.
3. Une sauvegarde en local des offres sont stockées dans un fichier (CSV ou Excel).

Les URL (FRANCE TRAVAIL) utiles sont :
- https://francetravail.io/data/api/offres-emploi
- https://francetravail.io/data/api/offres-emploi/documentation#/

Les URL (ADZUNA) utiles sont :
- https://developer.adzuna.com/overview
- https://developer.adzuna.com/docs/search
- https://developer.adzuna.com/activedocs#!/adzuna/search
- https://developer.adzuna.com/overview
- https://www.adzuna.fr/details/5376850320?utm_medium=api&utm_source=6d1ef246

## Imports

In [20]:
import http.client
import requests
import json
import pandas as pd
import os
import hashlib

## Procédure

### Configuration

In [21]:
##################  VARIABLES  ##################
# France Travail
FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
FT_SCOPE = os.environ.get("FT_SCOPE")

# Adzuna
ADZUNA_CLIENT_ID = os.environ.get("ADZUNA_CLIENT_ID")
ADZUNA_CLIENT_SECRET = os.environ.get("ADZUNA_CLIENT_SECRET")

### Paramètres de recherche

In [22]:
# ---------------------------
# PARAMETRES
# ---------------------------

# Paramètres de recherche
JOB_QUERY = "data analyst"
COMMUNE = "78300"
DISTANCE = 100000

# Nombre d'annonces par page requise
BLOC_PAGINATION = 50

# Nombre de pages max
MAX_PAGES = 20   # Limiter le nombre de pages récupérées

### Authentification France Travail

In [23]:
# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

### Lancement requête API France Travail

In [24]:
# ---------------------------
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_jobs(token, max_pages=MAX_PAGES):
    headers = {"Authorization": f"Bearer {token}"}
    all_jobs = []
    b_stop_criteria = False
    
    for page in range(1, max_pages + 1):
        if b_stop_criteria == False:    
            url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
            params = {
                "motsCles": JOB_QUERY,
                "commune": COMMUNE,
                "distance" : DISTANCE,
                "range": f"{(page-1)*BLOC_PAGINATION}-{page*BLOC_PAGINATION-1}"  # pagination par blocs de 50
            }
            r = requests.get(url, headers=headers, params=params)
            r.raise_for_status()
            data = r.json()
            offres = data.get("resultats", [])
                
            for o in offres:
                all_jobs.append({
                    "source": "France Travail",
                    "id":o.get("id"),                    
                    "titre": o.get("intitule"),
                    "description": o.get("description"),
                    "date": o.get("dateCreation"),
                    "date_actualisation": o.get("dateActualisation"),
                    "lieu_libelle": o.get("lieuTravail", {}).get("libelle"),
                    "lieu_latitude": o.get("lieuTravail", {}).get("latitude"),
                    "lieu_longitude": o.get("lieuTravail", {}).get("longitude"),
                    "lieu_codePostal": o.get("lieuTravail", {}).get("codePostal"),
                    "lieu_commune": o.get("lieuTravail", {}).get("commune"),
                    "entreprise_nom": o.get("entreprise", {}).get("nom"),
                    "entreprise_description": o.get("entreprise", {}).get("description"),
                    "entreprise_logo": o.get("entreprise", {}).get("logo"),
                    "entreprise_url": o.get("entreprise", {}).get("url"),
                    "typeContrat": o.get("typeContrat"),
                    "typeContratLibelle": o.get("typeContratLibelle"),
                    "natureContrat": o.get("natureContrat"),
                    "experienceExige": o.get("experienceExige"),
                    "experienceLibelle": o.get("experienceLibelle"),
                    "experienceCommentaire": o.get("experienceCommentaire"), 
                    
                    # "formations": o.get("formations"),
                    "formations_codeFormation": o.get("formations")[0].get("codeFormation") if o.get("formations") is not None else "None",
                    "formations_domaineLibelle": o.get("formations")[0].get("domaineLibelle") if o.get("formations") is not None else "None",
                    "formations_niveauLibelle": o.get("formations")[0].get("niveauLibelle") if o.get("formations") is not None else "None",
                    "formations_commentaire": o.get("formations")[0].get("commentaire") if o.get("formations") is not None else "None",
                    "formations_exigence": o.get("formations")[0].get("exigence") if o.get("formations") is not None else "None",
                    
                    # "langues": o.get("langues"),
                    "langues_libelle": o.get("langues")[0].get("libelle") if o.get("langues") is not None else "None",
                    "langues_exigence": o.get("langues")[0].get("exigence") if o.get("langues") is not None else "None",
                
                    "permis": o.get("permis"),
                    "permis_libelle": o.get("permis")[0].get("libelle") if o.get("permis") is not None else "None",
                    "permis_exigence": o.get("permis")[0].get("exigence") if o.get("permis") is not None else "None",

                    "outilsBureautiques": o.get("outilsBureautiques"),

                    # "competences": o.get("competences"),
                    "competences_code": o.get("competences")[0].get("code") if o.get("competences") is not None else "None",
                    "competences_libelle": o.get("competences")[0].get("libelle") if o.get("competences") is not None else "None",
                    "competences_exigence": o.get("competences")[0].get("exigence") if o.get("competences") is not None else "None",

                    # "salaire": o.get("salaire"),
                    "salaire_libelle": o.get("salaire").get("libelle") if o.get("salaire") is not None else "None",
                    "salaire_commentaire": o.get("salaire").get("commentaire") if o.get("salaire") is not None else "None",
                    "salaire_complement1": o.get("salaire").get("complement1") if o.get("salaire") is not None else "None",
                    "salaire_complement2": o.get("salaire").get("complement2") if o.get("salaire") is not None else "None",

                    "salaire_min" : o.get("salaire_min") if o.get("salaire_min") is not None else "None",
                    "salaire_max" : o.get("salaire_max") if o.get("salaire_max") is not None else "None",

                    "dureeTravailLibelle": o.get("dureeTravailLibelle"),

                    "dureeTravailLibelleConverti": o.get("dureeTravailLibelleConverti"),
                    "complementExercice": o.get("complementExercice"),
                    "conditionExercice": o.get("conditionExercice"),
                    "alternance": o.get("alternance"),

                    # "contact": o.get("contact"),
                    "contact_nom": o.get("contact").get("nom") if o.get("contact") is not None else "None",
                    "contact_coordonnees1": o.get("contact").get("coordonnees1") if o.get("contact") is not None else "None",
                    "contact_coordonnees2": o.get("contact").get("coordonnees2") if o.get("contact") is not None else "None",
                    "contact_coordonnees3": o.get("contact").get("coordonnees3") if o.get("contact") is not None else "None",
                    "contact_telephone": o.get("contact").get("telephone") if o.get("contact") is not None else "None",
                    "contact_courriel": o.get("contact").get("courriel") if o.get("contact") is not None else "None",
                    "contact_commentaire": o.get("contact").get("commentaire") if o.get("contact") is not None else "None",
                    "contact_urlRecruteur": o.get("contact").get("urlRecruteur") if o.get("contact") is not None else "None",
                    "contact_urlPostulation": o.get("contact").get("urlPostulation") if o.get("contact") is not None else "None",

                    # "agence": o.get("agence"),
                    "agence_telephone": o.get("agence").get("telephone") if o.get("agence") is not None else "None",
                    "agence_courriel": o.get("agence").get("courriel") if o.get("agence") is not None else "None",

                    "nombrePostes": o.get("nombrePostes"),
                    "deplacementCode": o.get("deplacementCode"),
                    "deplacementLibelle": o.get("deplacementLibelle"),

                    "qualificationCode": o.get("qualificationCode"),
                    "qualificationLibelle": o.get("qualificationLibelle"),

                    "codeNAF": o.get("codeNAF"),
                    "secteurActivite": o.get("secteurActivite"),
                    "secteurActiviteLibelle": o.get("secteurActiviteLibelle"),
                    
                    # "qualitesProfessionnelles": o.get("qualitesProfessionnelles"),
                    "qualitesProfessionnelles_libelle": o.get("qualitesProfessionnelles")[0].get("libelle") if o.get("qualitesProfessionnelles") is not None else "None",
                    "qualitesProfessionnelles_description": o.get("qualitesProfessionnelles")[0].get("description") if o.get("qualitesProfessionnelles") is not None else "None",

                    "trancheEffectifEtab": o.get("trancheEffectifEtab"),


                    # "origineOffre": o.get("origineOffre"),
                    "origineOffre_origine": o.get("origineOffre").get("origine") if o.get("origineOffre") is not None else "None",
                    "origineOffre_urlOrigine": o.get("origineOffre").get("urlOrigine") if o.get("origineOffre") is not None else "None",
                
                    # "origineOffre_partenaires": o.get("origineOffre").get("partenaires") if o.get("origineOffre") is not None else "None",

                    "origineOffre_partenaires_nom": o.get("origineOffre").get("partenaires")[0].get("nom") if o.get("origineOffre").get("partenaires") is not None else "None",
                    "origineOffre_partenaires_url": o.get("origineOffre").get("partenaires")[0].get("url") if o.get("origineOffre").get("partenaires") is not None else "None",
                    "origineOffre_partenaires_logo": o.get("origineOffre").get("partenaires")[0].get("logo") if o.get("origineOffre").get("partenaires") is not None else "None",

                    "offresManqueCandidats": o.get("offresManqueCandidats"),

                    # "contexteTravail": o.get("contexteTravail"),
                    "contexteTravail_horaires": o.get("contexteTravail").get("horaires")[0] if o.get("contexteTravail").get("horaires") is not None else "None",
                    "contexteTravail_conditionsExercice": o.get("contexteTravail").get("conditionsExercice")[0] if o.get("contexteTravail").get("conditionsExercice") is not None else "None",
                })
            print(len(offres))

            # Si le nombre d'offres est inférieur au nombre max d'offre par pages, c'est un signe qu'il n'y a plus d'offres à extraire après la page actuelle.
            if len(offres) < BLOC_PAGINATION:
                b_stop_criteria = True
            
    return all_jobs

### Lancement de requête Adzuna

In [25]:
# ---------------------------
# API CALL ADZUNA
# ---------------------------
def fetch_adzuna_jobs(max_pages=MAX_PAGES):
    headers = {"Accept": "application/json"}
    all_jobs = []
    b_stop_criteria = False
    
    for page in range(1, max_pages + 1):
        if b_stop_criteria == False:    
            url = f"https://api.adzuna.com/v1/api/jobs/fr/search/{page}"
            params = {
                "app_id" : ADZUNA_CLIENT_ID,
                "app_key" : ADZUNA_CLIENT_SECRET,
                "title_only": JOB_QUERY,
                "where": COMMUNE,
                "results_per_page" : BLOC_PAGINATION,
                "distance" : DISTANCE
            }
            r = requests.get(url,params=params)
            r.raise_for_status()
            data = r.json()
            offres = data.get("results")
                
            print(len(offres))

            for o in offres:
                all_jobs.append({
                    "source": "Adzuna",
                    "id" : o.get("id"),
                    "titre" : o.get("title"),
                    "description" : o.get("description"),
                    "date" : o.get("created"),
                    "date_actualisation": o.get("dateActualisation") if o.get("dateActualisation") is not None else "None",  
                    "lieu_libelle" : o.get("location").get("display_name") if o.get("location") is not None else "None",                  
                    "lieu_latitude" : o.get("latitude"),
                    "lieu_longitude" : o.get("longitude"),
                    "lieu_codePostal": o.get("lieu_codePostal") if o.get("lieu_codePostal") is not None else "None", 
                    "lieu_commune": o.get("lieu_commune") if o.get("lieu_commune") is not None else "None",              
                    "entreprise_nom": o.get("company").get("display_name") if o.get("company") is not None else "None",
                    "entreprise_description": o.get("entreprise_description") if o.get("entreprise_description") is not None else "None",  
                    "entreprise_logo": o.get("entreprise_logo") if o.get("entreprise_logo") is not None else "None",  
                    "entreprise_url": o.get("entreprise_url") if o.get("entreprise_url") is not None else "None",  

                    "typeContrat": o.get("typeContrat") if o.get("typeContrat") is not None else "None",  
                    "typeContratLibelle" : o.get("contract_type"),

                    
                    "natureContrat": o.get("natureContrat") if o.get("natureContrat") is not None else "None",  
                    "experienceExige": o.get("experienceExige") if o.get("experienceExige") is not None else "None",  
                    "experienceLibelle": o.get("experienceLibelle") if o.get("experienceLibelle") is not None else "None",  
                    "experienceCommentaire": o.get("experienceCommentaire") if o.get("experienceCommentaire") is not None else "None",  
   
                    # "formations": o.get("formations"),
                    "formations_codeFormation": o.get("formations_codeFormation") if o.get("formations_codeFormation") is not None else "None",  
                    "formations_domaineLibelle": o.get("formations_domaineLibelle") if o.get("formations_domaineLibelle") is not None else "None",  
                    "formations_niveauLibelle": o.get("formations_niveauLibelle") if o.get("formations_niveauLibelle") is not None else "None",  
                    "formations_commentaire": o.get("formations_commentaire") if o.get("formations_commentaire") is not None else "None",  
                    "formations_exigence": o.get("formations_exigence") if o.get("formations_exigence") is not None else "None",  


                    # "langues": o.get("langues"),
                    "langues_libelle": o.get("formations_exigence") if o.get("formations_exigence") is not None else "None", 
                    "langues_exigence": o.get("formations_exigence") if o.get("formations_exigence") is not None else "None", 
                
                    "permis": o.get("permis") if o.get("permis") is not None else "None", 
                    "permis_libelle": o.get("permis_libelle") if o.get("permis_libelle") is not None else "None", 
                    "permis_exigence": o.get("permis_exigence") if o.get("permis_exigence") is not None else "None", 

                    "outilsBureautiques": o.get("outilsBureautiques") if o.get("outilsBureautiques") is not None else "None", 

                    # "competences": o.get("competences") if o.get("competences") is not None else "None",
                    "competences_code": o.get("competences_code") if o.get("competences_code") is not None else "None",
                    "competences_libelle": o.get("competences_libelle") if o.get("competences_libelle") is not None else "None",
                    "competences_exigence": o.get("competences_exigence") if o.get("competences_exigence") is not None else "None",

                    # "salaire": o.get("salaire") if o.get("salaire") is not None else "None",
                    "salaire_libelle": o.get("salaire_libelle") if o.get("salaire_libelle") is not None else "None",
                    "salaire_commentaire": o.get("salaire_commentaire") if o.get("salaire_commentaire") is not None else "None",
                    "salaire_complement1": o.get("salaire_complement1") if o.get("salaire_complement1") is not None else "None",
                    "salaire_complement2": o.get("salaire_complement2") if o.get("salaire_complement2") is not None else "None",
                    "salaire_min" : o.get("salary_min"),
                    "salaire_max" : o.get("salary_max"),


                    "dureeTravailLibelle": o.get("dureeTravailLibelle") if o.get("dureeTravailLibelle") is not None else "None",

                    "dureeTravailLibelleConverti": o.get("dureeTravailLibelleConverti") if o.get("dureeTravailLibelleConverti") is not None else "None",
                    "complementExercice": o.get("complementExercice") if o.get("complementExercice") is not None else "None",
                    "conditionExercice": o.get("conditionExercice") if o.get("conditionExercice") is not None else "None",
                    "alternance": o.get("alternance") if o.get("alternance") is not None else "None",

                    # "contact": o.get("contact") if o.get("contact") is not None else "None",
                    "contact_nom": o.get("contact_nom") if o.get("contact_nom") is not None else "None",
                    "contact_coordonnees1": o.get("contact_coordonnees1") if o.get("contact_coordonnees1") is not None else "None",
                    "contact_coordonnees2": o.get("contact_coordonnees2") if o.get("contact_coordonnees2") is not None else "None",
                    "contact_coordonnees3": o.get("contact_coordonnees3") if o.get("contact_coordonnees3") is not None else "None",
                    "contact_telephone": o.get("contact_telephone") if o.get("contact_telephone") is not None else "None",
                    "contact_courriel": o.get("contact_courriel") if o.get("contact_courriel") is not None else "None",
                    "contact_commentaire": o.get("contact_commentaire") if o.get("contact_commentaire") is not None else "None",
                    "contact_urlRecruteur": o.get("contact_urlRecruteur") if o.get("contact_urlRecruteur") is not None else "None",
                    "contact_urlPostulation": o.get("contact_urlPostulation") if o.get("contact_urlPostulation") is not None else "None",

                    # "agence": o.get("agence") if o.get("agence") is not None else "None",
                    "agence_telephone": o.get("agence_telephone") if o.get("agence_telephone") is not None else "None",
                    "agence_courriel": o.get("agence_courriel") if o.get("agence_courriel") is not None else "None",

                    "nombrePostes": o.get("nombrePostes") if o.get("nombrePostes") is not None else "None",
                    "deplacementCode": o.get("deplacementCode") if o.get("deplacementCode") is not None else "None",
                    "deplacementLibelle": o.get("deplacementLibelle") if o.get("deplacementLibelle") is not None else "None",

                    "qualificationCode": o.get("qualificationCode") if o.get("qualificationCode") is not None else "None",
                    "qualificationLibelle": o.get("qualificationLibelle") if o.get("qualificationLibelle") is not None else "None",

                    "codeNAF": o.get("codeNAF") if o.get("codeNAF") is not None else "None",
                    "secteurActivite": o.get("secteurActivite") if o.get("secteurActivite") is not None else "None",
                    "secteurActiviteLibelle" : o.get("category").get("label") if o.get("category") is not None else "None",

                    # "qualitesProfessionnelles": o.get("qualitesProfessionnelles") if o.get("qualitesProfessionnelles") is not None else "None",
                    "qualitesProfessionnelles_libelle": o.get("qualitesProfessionnelles_libelle") if o.get("qualitesProfessionnelles_libelle") is not None else "None",
                    "qualitesProfessionnelles_description": o.get("qualitesProfessionnelles_description") if o.get("qualitesProfessionnelles_description") is not None else "None",

                    "trancheEffectifEtab": o.get("trancheEffectifEtab") if o.get("trancheEffectifEtab") is not None else "None",
                    

                    # "origineOffre": o.get("origineOffre") if o.get("origineOffre") is not None else "None",
                    "origineOffre_origine": o.get("origineOffre_origine") if o.get("origineOffre_origine") is not None else "None",
                    "origineOffre_urlOrigine" : o.get("redirect_url"),     
                
                    # "origineOffre_partenaires": o.get("origineOffre_partenaires") if o.get("origineOffre_partenaires") is not None else "None",

                    "origineOffre_partenaires_nom": o.get("origineOffre_partenaires_nom") if o.get("origineOffre_partenaires_nom") is not None else "None",
                    "origineOffre_partenaires_url": o.get("origineOffre_partenaires_url") if o.get("origineOffre_partenaires_url") is not None else "None",
                    "origineOffre_partenaires_logo": o.get("origineOffre_partenaires_logo") if o.get("origineOffre_partenaires_logo") is not None else "None",

                    "offresManqueCandidats": o.get("offresManqueCandidats") if o.get("offresManqueCandidats") is not None else "None",

                    # "contexteTravail": o.get("contexteTravail") if o.get("contexteTravail") is not None else "None",
                    "contexteTravail_horaires": o.get("contexteTravail_horaires") if o.get("contexteTravail_horaires") is not None else "None",
                    "contexteTravail_conditionsExercice": o.get("contexteTravail_conditionsExercice") if o.get("contexteTravail_conditionsExercice") is not None else "None",
    
                                           
                })
                
            # Si le nombre d'offres est inférieur au nombre max d'offre par pages, c'est un signe qu'il n'y a plus d'offres à extraire après la page actuelle.
            if len(offres) < BLOC_PAGINATION:
                b_stop_criteria = True
            
    return all_jobs

### Déduplication

In [26]:
# ---------------------------
# DÉDUPLICATION
# ---------------------------
def deduplicate(jobs):
    seen = set()
    deduped = []
    for job in jobs:
        key_str = f"{job['titre']}_{job['entreprise_nom']}_{job['lieu_latitude']}_{job['lieu_longitude']}_{job['date']}"
        key = hashlib.md5(key_str.encode()).hexdigest()
        if key not in seen:
            seen.add(key)
            deduped.append(job)
    return deduped

### Procédure principale

In [27]:
# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    print("Authentification France Travail...")
    token = get_ft_token()

    print("Récupération des offres France Travail...")
    ft_jobs = fetch_france_travail_jobs(token)

    print("Récupération des offres Adzuna...")
    adzuna_jobs = fetch_adzuna_jobs()

    print("Fusion et déduplication...")
    all_jobs = ft_jobs + adzuna_jobs

    print(f"Nombre d'offres d'emploi avant déduplication : {len(all_jobs)}")
    jobs_clean = deduplicate(all_jobs)
    print(f"Nombre d'offres d'emploi après déduplication : {len(jobs_clean)}")

    print("Affichage Extract offres France Travail...")
    df = pd.DataFrame(jobs_clean)
    display(df.shape)
    display(df.head())

    # Export vers CSV
    path = "../data/raw_data/"
    file_name = "offres_emploi_france_travail_adzuna.csv"
    
    df.to_csv(path + file_name, index=False, encoding="utf-8")
    print(f"{len(jobs_clean)} offres uniques exportées dans {file_name} ✅")

Authentification France Travail...
Récupération des offres France Travail...
50
50
50
33
Récupération des offres Adzuna...
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
Fusion et déduplication...
Nombre d'offres d'emploi avant déduplication : 1183
Nombre d'offres d'emploi après déduplication : 1181
Affichage Extract offres France Travail...


(1181, 76)

,source,id,titre,description,date,date_actualisation,lieu_libelle,lieu_latitude,lieu_longitude,lieu_codePostal,...,qualitesProfessionnelles_description,trancheEffectifEtab,origineOffre_origine,origineOffre_urlOrigine,origineOffre_partenaires_nom,origineOffre_partenaires_url,origineOffre_partenaires_logo,offresManqueCandidats,contexteTravail_horaires,contexteTravail_conditionsExercice
0,France Travail,197NKZD,Consultant Data confirmé - Data Analyst H/F,Nous recherchons un(e) Consultant(e) Data conf...,2025-09-09T09:24:40.097Z,2025-09-15T14:22:01.346Z,49 - TRELAZE,47.445931,-0.466974,49800,...,None,None,1,https://candidat.francetravail.fr/offres/reche...,None,None,None,False,35H/semaine\nTravail en journée,None
1,France Travail,197PLRS,Data Analyst (h/f),"Adecco recherche pour l'un de ses clients, une...",2025-09-09T16:27:22.779Z,2025-09-10T08:31:13.461Z,13 - Marignane,43.407483,5.255358,13700,...,None,None,1,https://candidat.francetravail.fr/offres/reche...,None,None,None,False,35H/semaine\nTravail en journée,None
2,France Travail,197JSYL,Data analyst (F/H) (H/F),"Au sein de la direction Excellence Clients, no...",2025-09-04T13:51:37.276Z,2025-09-08T10:46:32.157Z,26 - Valence,44.933273,4.892158,26000,...,"Capacité à planifier, prioriser, anticiper des...",None,1,https://candidat.francetravail.fr/offres/reche...,None,None,None,False,39H/semaine\nTravail en journée,None
3,France Travail,197FWLH,Data analyst senior (H/F),Data Analyst Senior F/H - CDI - La Défense\n\n...,2025-09-02T11:49:20.769Z,2025-09-03T10:44:07.803Z,92 - PUTEAUX,48.884019,2.237979,92800,...,None,None,1,https://candidat.francetravail.fr/offres/reche...,None,None,None,False,36H30/semaine\nTravail en journée,Station assise prolongée
4,France Travail,197GFLX,Data analyst (F/H),"Directement rattaché au DSI , vous serez un él...",2025-09-02T14:41:39.561Z,2025-09-02T14:41:39.755Z,34 - Montpellier,43.626378,3.882533,34000,...,None,None,1,https://candidat.francetravail.fr/offres/reche...,None,None,None,False,35H/semaine\nTravail en journée,None


1181 offres uniques exportées dans offres_emploi_france_travail_adzuna.csv ✅
